In [16]:
import numpy as np
import pandas as pd
import numpy.random as npr
import json
from IPython.display import display # to print nicely formatted dataframe in a more consistent way

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [7]:
def convertToCSV(srcFileName, destFileName):
    with open(srcFileName, 'r') as inFile, open(destFileName, 'w') as outFile:
        line = inFile.read()
        line = line.replace(" ", ",")
        outFile.write(line)

# create the list
anIntList = [x for x in range(10)]
aFloatList = np.linspace(0,1,25).reshape(5,5)
intFileName = "./data/data_int.txt"
floatFileName = "./data/data_float.txt"

# saving to file
with open("./data/data_int.txt", 'w') as outFile:
    # separate case for the first N-1 numbers and the last one because I don't want a
    # trailing space at the end of the sequence.
    for n in range(len(anIntList)-1): outFile.write(str(anIntList[n]) + ' ')
    outFile.write(str(anIntList[-1]))
    
with open(floatFileName, 'w') as outFile:
    for n in range(len(aFloatList)):
        for m in range(len(aFloatList[n]) -1):
            outFile.write(str(aFloatList[n][m]) + ' ')
        outFile.write(str(aFloatList[n][-1]))
        outFile.write("\n")

# printing the two files
print("integer list from file:")
!cat ./data/data_int.txt
print()
print("\nfloat matrix from file:")
!cat ./data/data_float.txt

# converting to CSV and printing the results
convertToCSV(intFileName, "data/data_int.csv")
convertToCSV(floatFileName, "data/data_float.csv")
print("\ninteger CSV from file:")
!cat ./data/data_int.csv
print()
print("\nfloat CSV from file:")
!cat ./data/data_float.csv

integer list from file:
0 1 2 3 4 5 6 7 8 9

float matrix from file:
0.0 0.041666666666666664 0.08333333333333333 0.125 0.16666666666666666
0.20833333333333331 0.25 0.29166666666666663 0.3333333333333333 0.375
0.41666666666666663 0.4583333333333333 0.5 0.5416666666666666 0.5833333333333333
0.625 0.6666666666666666 0.7083333333333333 0.75 0.7916666666666666
0.8333333333333333 0.875 0.9166666666666666 0.9583333333333333 1.0

integer CSV from file:
0,1,2,3,4,5,6,7,8,9

float CSV from file:
0.0,0.041666666666666664,0.08333333333333333,0.125,0.16666666666666666
0.20833333333333331,0.25,0.29166666666666663,0.3333333333333333,0.375
0.41666666666666663,0.4583333333333333,0.5,0.5416666666666666,0.5833333333333333
0.625,0.6666666666666666,0.7083333333333333,0.75,0.7916666666666666
0.8333333333333333,0.875,0.9166666666666666,0.9583333333333333,1.0


2\. **JSON files**

Load the file `user_data.json`, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [17]:
# !wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json -P ./data
#!cat data/user_data.json
with open('./data/user_data.json', 'r') as f:
    data= pd.read_json(f)
amex = data[data['CreditCardType'] == "American Express"]
amex.to_csv('./data/amex_customers.csv')
display(amex)
#!cat ./data/amex_customers.csv

,ID,JobTitle,EmailAddress,FirstNameLastName,CreditCard,CreditCardType
1,2,Investment Advisor,Clint_Thorpe5003@bulaffy.com,Clint Thorpe,7083-8766-0251-2345,American Express
11,12,Retail Trainee,Phillip_Carpenter9505@famism.biz,Phillip Carpenter,3657-0088-0820-5247,American Express
27,28,Project Manager,Russel_Graves1378@extex.org,Russel Graves,6718-4818-8011-6024,American Express
38,39,Stockbroker,Leanne_Newton1268@typill.biz,Leanne Newton,5438-0816-4166-4847,American Express
56,57,Budget Analyst,Tony_Giles1960@iatim.tech,Tony Giles,8130-3425-7573-7745,American Express
61,62,CNC Operator,Owen_Allcott5125@bauros.biz,Owen Allcott,4156-0107-7210-2630,American Express
67,68,Project Manager,Liam_Lynn3280@kideod.biz,Liam Lynn,7152-3247-6053-2233,American Express
73,74,Dentist,Regina_Woodcock5820@yahoo.com,Regina Woodcock,0208-1753-3870-8002,American Express
80,81,HR Specialist,Carter_Wallace9614@atink.com,Carter Wallace,4256-7201-6717-4322,American Express
91,92,Staffing Consultant,Maia_Stark2797@jiman.org,Maia Stark,3851-1403-1734-6321,American Express


3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [19]:
# !wget https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv -P data/
with open("data/mushrooms_categorized.csv", 'r') as fin:
    data = pd.read_csv(fin)
display(data)
data = data.groupby('class').mean()
data.to_json("data/mushroom_recap.json")
display(data)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
class,,,,,,,,,,,,,,,,,,,,,
0,3.266160,1.615970,4.581749,0.653992,4.334601,0.954373,0.285171,0.068441,6.622624,0.615970,...,1.798479,6.098859,6.064639,0.0,1.931559,1.125475,3.007605,3.201521,3.283270,1.148289
1,3.436159,2.055158,4.421859,0.159346,3.940756,0.995403,0.028601,0.567926,2.863636,0.514811,...,1.394280,5.512768,5.504597,0.0,2.002043,1.009193,1.522983,4.021450,4.031665,1.895812


4\. **Reading a database**

Get the database `sakila.db` from the lecture `06_dataio.ipynb`, and import the table `actors` as a Pandas dataframe. Using the dataframe, count how many actors have a first name that begins with `A`.

*Hint:* use the Series `.str` method to apply the Python string methods to the elements of a Series, see [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.html).

In [26]:
#!wget https://gist.github.com/Piyush3dB/726bf7012785d6e0fd691c3655c92654/raw/2c17ccb2eb33b3396bfa96284c53f0718a4ea62c/sakila.db -P ./data/
# import dependency
import sqlite3 as sql

# create a connection to the database and a cursor to execute queries
conn = sql.connect('data/sakila.db')
cur = conn.cursor()

# query data from database: select all content from the table "actor"
query = "SELECT * FROM actor"
results = cur.execute(query).fetchall()

# create a DataFrame from the DB data
df = pd.DataFrame(results)

# prepare the mask
mask = df[1].str
mask = mask[0] == 'A'

# close the cursor and connection
cur.close()
conn.close()

# print dataframe with the mask
entries = df[mask].shape[0]
print("the number of actors whose first name starts with A is", entries)
display(df[mask])



the number of actors whose first name starts with A is 13


,0,1,2,3
28,29,ALEC,WAYNE,2019-02-16 18:17:33
33,34,AUDREY,OLIVIER,2019-02-16 18:17:33
48,49,ANNE,CRONYN,2019-02-16 18:17:33
64,65,ANGELA,HUDSON,2019-02-16 18:17:33
70,71,ADAM,GRANT,2019-02-16 18:17:33
75,76,ANGELINA,ASTAIRE,2019-02-16 18:17:33
124,125,ALBERT,NOLTE,2019-02-16 18:17:33
131,132,ADAM,HOPPER,2019-02-16 18:17:33
143,144,ANGELA,WITHERSPOON,2019-02-16 18:17:33
145,146,ALBERT,JOHANSSON,2019-02-16 18:17:33


5\. **Reading the credit card numbers**

Get the binary file named `credit_card.dat` from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [12]:
# !wget https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat -P data/
#!hexdump data/credit_card.dat

with open('data/credit_card.dat', 'rb') as fin:
    i = 1
    current = ""
    while True:
        if(i == 20):
            fin.read(5)
            print(current)
            current = ""
            i = 1
        else:
            chunk = fin.read(6)
            if len(chunk) == 0:
                break
            current = current + str(chr(int(chunk, 2)))
            i += 1



--2023-11-24 09:21:02--  https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/8m0syw2tkul3dap/credit_card.dat [following]
--2023-11-24 09:21:02--  https://www.dropbox.com/s/raw/8m0syw2tkul3dap/credit_card.dat
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc26d62a547cdb66741abe0df661.dl.dropboxusercontent.com/cd/0/inline/CIJQ18g7hMI6FUCTCnS-_OEtBcbz4HY2-r09uMel4oqwipI4NfRw-MkWNLI2B7hX2TVfFOI6gOhY7snG2aaQU_jG84vjnw1eWpDthqVEPcntyEUm7Ab9rssZm5Po2Ws-90E/file# [following]
--2023-11-24 09:21:03--  https://uc26d62a547cdb66741abe0df661.dl.dropboxusercontent.com/cd/0/inline/CIJQ18g7hMI6FUCTCnS-_OEtBcbz4HY2-r09uMel4oqwipI4NfRw-MkWNLI2B7hX2TVfFOI6gOhY7snG2aaQU_jG84vjnw1eWpDthqVEPcntyEUm

6\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, "pack" the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_dataio.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?

In [28]:
#!wget https://www.dropbox.com/s/ga9wi6b40cakgae/data_000637.txt -P data/

import struct


dataIn = pd.read_csv('./data/data_000637.txt', nrows=10)
print("the original text file reads as follows: \n")
print(dataIn)

# writing the data as text file for comparison
with open('data/text_file.txt', mode='w') as ftxt:
            ftxt.write(str(dataIn))

# writing as binary file
with open('data/binary_file.dat','wb')as fbin:
     for line in dataIn.values:
        word = line[0] << 62   #index
        word += line[1] << 58  #FPGA number
        word += line[2] << 49  #TDC channel
        word += line[3] << 17  #orbit CNT
        word += line[4] << 5   #BX counter
        word += line[5] << 0   #TDC meas
        fbin.write(struct.pack('<q',word))

# this is the code from the lesson to read the data file, it should (and does) have the same output as
# the one above

import struct, time

data = {}

with open('data/binary_file.dat', 'rb') as file:
    file_content = file.read()
    word_counter = 0
    word_size = 8 # size of the word in bytes
    for i in range(0, len(file_content), word_size):
        word_counter += 1
        word = struct.unpack('<q', file_content[i : i + word_size])[0] # get an 8-byte word
        head     = (word >> 62) & 0x3
        fpga     = (word >> 58) & 0xF
        tdc_chan = (word >> 49) & 0x1FF
        orb_cnt  = (word >> 17) & 0xFFFFFFFF
        bx       = (word >> 5 ) & 0xFFF
        tdc_meas = (word >> 0 ) & 0x1F
        #if i == 0: print ('{0}\t{1}\t{2}\t{3}\t{4}\t{5}'.format('HEAD', 'FPGA', 'CHANNEL', 'ORBIT_CNT', 'BX_CNT', 'TDC_MEAS'))
        #print('{0}\t{1}\t{2}\t{3}\t{4}\t{5}'.format(head, fpga, tdc_chan, orb_cnt, bx, tdc_meas))
        entry = {'HEAD' : head, 'FPGA' : fpga, 'CHANNEL' : tdc_chan, 'ORBIT_CNT' : orb_cnt, 'BX_CNT' : bx, 'TDC_MEAS' : tdc_meas}
        #df = df.append(entry, ignore_index=True)
        data[word_counter] = entry

print("\n\nsaving the file as binary and subsequently reading it with the code presented during the lecture, we obtain:\n")
df = pd.DataFrame(data).T
display(df)

print("\nas we can see, both files read correctly.")


the original text file reads as follows: 

   HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0     1     0          123  3869200167        2374        26
1     1     0          124  3869200167        2374        27
2     1     0           63  3869200167        2553        28
3     1     0           64  3869200167        2558        19
4     1     0           64  3869200167        2760        25
5     1     0           63  3869200167        2762         4
6     1     0           61  3869200167        2772        14
7     1     0          139  3869200167        2776         0
8     1     0           62  3869200167        2774        21
9     1     0           60  3869200167        2788         7


saving the file as binary and subsequently reading it with the code presented during the lecture, we obtain:



,HEAD,FPGA,CHANNEL,ORBIT_CNT,BX_CNT,TDC_MEAS
1,1,0,123,3869200167,2374,26
2,1,0,124,3869200167,2374,27
3,1,0,63,3869200167,2553,28
4,1,0,64,3869200167,2558,19
5,1,0,64,3869200167,2760,25
6,1,0,63,3869200167,2762,4
7,1,0,61,3869200167,2772,14
8,1,0,139,3869200167,2776,0
9,1,0,62,3869200167,2774,21
10,1,0,60,3869200167,2788,7



as we can see, both files read correctly.


In [29]:
from pathlib import Path
print("text file size:",Path('data/text_file.txt').stat().st_size, 'B')
print("bin file size:",Path('data/binary_file.dat').stat().st_size, 'B')

text file size: 670 B
bin file size: 80 B
